#### REGULARIZATION

We want to create two prediction models both for solar energy output and wind energy output. In order to predict these outputs we want to build a model that selects the best possible predictors maximizing the feature significance, avoiding colinearity, ensuring sparisty, considering the best feature transformations... To do so we will apply different models like Lasso, and Ridge and then we can apply Holistic Regression to see how the model is improved by it. We will run all of these models by all predictors of solar and energy output. 

1. SOLAR ENERGY OUTPUT

In [ ]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics, DataFrames
#read train data and split it into X and y
